In [55]:
import weaviate
from weaviate.classes.config import Configure, Property, DataType

client = weaviate.connect_to_local(
    port=7575
)

client.collections.delete_all()

# Note that you can use `client.collections.create_from_dict()` to create a collection from a v3-client-style JSON object
client.collections.create(
    "News",
    vectorizer_config= Configure.Vectorizer.text2vec_transformers(),#inference_url="http://localhost:7474"),
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="header", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
    ]
)

In [56]:
## Add a some news

news_collection = client.collections.get("News")
new_berry = news_collection.data.insert(
    properties={
        "title": "Berry Global invests to increase production capacity for healthcare products",
        "header": "",
        "body": """Berry Global Group, Inc., a leader in the design, development, and production of patient-centred healthcare solutions, has invested in additional assets and manufacturing capabilities to increase its healthcare production capacity by up to 30% cross three of its European sites.
Following the opening of its manufacturing facility and healthcare centre of excellence in Bangalore, India, Berry says these latest investments underline its commitment to delivering high-quality packaging and drug delivery devices that enhance the patient experience through improved medicine administration and adherence.
“As pharmaceutical innovations progress, so should the packaging and support that delivers each dose. Berry Global is here to provide that expertise in the creation of patient-centred packages and drug delivery device design,” said Walid Hamza, EVP Global Healthcare at Berry Global. “We continue to invest in the latest healthcare technologies to help our customers offer patients a safe and improved experience with every dose.”""",
    },
    #references={"ticker": "BERY"},
)
new_tesla = news_collection.data.insert(
    properties={
        "title": "Protesters attempt to storm Tesla’s factory in Germany",
        "header": "As many as 800 activists gathered outside Tesla’s factory near Berlin Friday to protest its expansion plans, and some of them clashed with police as they attempted to break into the plant.",
        "body": """But Tesla CEO Elon Musk wrote in a post on X Friday: “Protesters did not manage to break through the fence line. There are still two intact fence lines all around (the factory).”
Disrupt argues that Musk’s plans to more than double the production capacity of Tesla’s only factory in Europe would damage the local environment.
The group says the expansion would require clearing swathes of the surrounding forest and would further strain local water supply. It has planned four days of protests, which started Wednesday.”""",
    },
    #references={"ticker": "TSLA"},
)

In [58]:
news_collection = client.collections.get("News")

response = news_collection.query.fetch_objects(    
    include_vector=True,
)

for o in response.objects:
    print(o.properties)
    print(o.vector)

{'title': 'Berry Global invests to increase production capacity for healthcare products', 'header': '', 'body': 'Berry Global Group, Inc., a leader in the design, development, and production of patient-centred healthcare solutions, has invested in additional assets and manufacturing capabilities to increase its healthcare production capacity by up to 30% cross three of its European sites.\nFollowing the opening of its manufacturing facility and healthcare centre of excellence in Bangalore, India, Berry says these latest investments underline its commitment to delivering high-quality packaging and drug delivery devices that enhance the patient experience through improved medicine administration and adherence.\n“As pharmaceutical innovations progress, so should the packaging and support that delivers each dose. Berry Global is here to provide that expertise in the creation of patient-centred packages and drug delivery device design,” said Walid Hamza, EVP Global Healthcare at Berry Globa

In [68]:
response = news_collection.query.bm25(
    query="Protesters",
    limit=2
)
for o in response.objects:
    print(o.properties)


{'title': 'Protesters attempt to storm Tesla’s factory in Germany', 'header': 'As many as 800 activists gathered outside Tesla’s factory near Berlin Friday to protest its expansion plans, and some of them clashed with police as they attempted to break into the plant.', 'body': 'But Tesla CEO Elon Musk wrote in a post on X Friday: “Protesters did not manage to break through the fence line. There are still two intact fence lines all around (the factory).”\nDisrupt argues that Musk’s plans to more than double the production capacity of Tesla’s only factory in Europe would damage the local environment.\nThe group says the expansion would require clearing swathes of the surrounding forest and would further strain local water supply. It has planned four days of protests, which started Wednesday.”'}


In [70]:
response = news_collection.query.bm25(
    query="Thread",
    limit=2
)
for o in response.objects:
    print(f"Thread occurrence: {o.properties}")

response = news_collection.query.bm25(
    query="Production",
    limit=2
)
for o in response.objects:
    print(f"Production occurrence: {o.properties}")

Production occurrence: {'title': 'Berry Global invests to increase production capacity for healthcare products', 'header': '', 'body': 'Berry Global Group, Inc., a leader in the design, development, and production of patient-centred healthcare solutions, has invested in additional assets and manufacturing capabilities to increase its healthcare production capacity by up to 30% cross three of its European sites.\nFollowing the opening of its manufacturing facility and healthcare centre of excellence in Bangalore, India, Berry says these latest investments underline its commitment to delivering high-quality packaging and drug delivery devices that enhance the patient experience through improved medicine administration and adherence.\n“As pharmaceutical innovations progress, so should the packaging and support that delivers each dose. Berry Global is here to provide that expertise in the creation of patient-centred packages and drug delivery device design,” said Walid Hamza, EVP Global He

In [65]:
# production in both news
# thread in no new

response = news_collection.query.near_text(
    query="Threads in production",
    limit=1,
)
for o in response.objects:
    print(o.properties)


{'title': 'Protesters attempt to storm Tesla’s factory in Germany', 'header': 'As many as 800 activists gathered outside Tesla’s factory near Berlin Friday to protest its expansion plans, and some of them clashed with police as they attempted to break into the plant.', 'body': 'But Tesla CEO Elon Musk wrote in a post on X Friday: “Protesters did not manage to break through the fence line. There are still two intact fence lines all around (the factory).”\nDisrupt argues that Musk’s plans to more than double the production capacity of Tesla’s only factory in Europe would damage the local environment.\nThe group says the expansion would require clearing swathes of the surrounding forest and would further strain local water supply. It has planned four days of protests, which started Wednesday.”'}


In [66]:
response = news_collection.query.near_text(
    query="Good news",
    limit=1,
)
for o in response.objects:
    print(o.properties)


{'title': 'Berry Global invests to increase production capacity for healthcare products', 'header': '', 'body': 'Berry Global Group, Inc., a leader in the design, development, and production of patient-centred healthcare solutions, has invested in additional assets and manufacturing capabilities to increase its healthcare production capacity by up to 30% cross three of its European sites.\nFollowing the opening of its manufacturing facility and healthcare centre of excellence in Bangalore, India, Berry says these latest investments underline its commitment to delivering high-quality packaging and drug delivery devices that enhance the patient experience through improved medicine administration and adherence.\n“As pharmaceutical innovations progress, so should the packaging and support that delivers each dose. Berry Global is here to provide that expertise in the creation of patient-centred packages and drug delivery device design,” said Walid Hamza, EVP Global Healthcare at Berry Globa